This project involves retrieving and organizing data from the NHL's public API.  lists of player ID numbers with which they want to be able to programmatically pull their statistics and output them in a spreadsheet.

In [3]:
import pandas as pd

Import our Excel file of ID numers below.

In [4]:
excel_file = 'pythonproject.xlsx'
nhl = pd.read_excel(excel_file)

In [5]:
nhl.head()

,playerID,url
0,8470600,http://statsapi.web.nhl.com/api/v1/people/8470...
1,8475218,http://statsapi.web.nhl.com/api/v1/people/8475...
2,8471709,http://statsapi.web.nhl.com/api/v1/people/8471...
3,8476958,http://statsapi.web.nhl.com/api/v1/people/8476...
4,8474031,http://statsapi.web.nhl.com/api/v1/people/8474...


The way to retrieve the data from the URL is using Python's 'requests' library. Use 'response.get()' on a URL to receive the data, then use the built-in json() function to automatically return the response in JSON format.

In [6]:
import requests

In [7]:
example_response = requests.get(nhl["url"][0])
example_response.json()

{'copyright': 'NHL and the NHL Shield are registered trademarks of the National Hockey League. NHL and NHL team marks are the property of the NHL and its teams. © NHL 2018. All Rights Reserved.',
 'people': [{'active': True,
   'alternateCaptain': True,
   'birthCity': 'Madison',
   'birthCountry': 'USA',
   'birthDate': '1985-01-21',
   'birthStateProvince': 'WI',
   'captain': False,
   'currentAge': 33,
   'currentTeam': {'id': 30,
    'link': '/api/v1/teams/30',
    'name': 'Minnesota Wild'},
   'firstName': 'Ryan',
   'fullName': 'Ryan Suter',
   'height': '6\' 2"',
   'id': 8470600,
   'lastName': 'Suter',
   'link': '/api/v1/people/8470600',
   'nationality': 'USA',
   'primaryNumber': '20',
   'primaryPosition': {'abbreviation': 'D',
    'code': 'D',
    'name': 'Defenseman',
    'type': 'Defenseman'},
   'rookie': False,
   'rosterStatus': 'Y',
   'shootsCatches': 'L',
   'stats': [{'splits': [{'season': '20182019',
       'stat': {'assists': 5,
        'blocked': 18,
        

JSON format is nice but we will need to convert it to a flat table if we want to be able to output it to a spreadsheet. json_normalize will convert the data into a one-row Pandas dataframe.

In [8]:
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', 500)  # just for Jupyter display purposes 

example_row = json_normalize(example_response.json()['people'])
example_row

,active,alternateCaptain,birthCity,birthCountry,birthDate,birthStateProvince,captain,currentAge,currentTeam.id,currentTeam.link,currentTeam.name,firstName,fullName,height,id,lastName,link,nationality,primaryNumber,primaryPosition.abbreviation,primaryPosition.code,primaryPosition.name,primaryPosition.type,rookie,rosterStatus,shootsCatches,stats,weight
0,True,True,Madison,USA,1985-01-21,WI,False,33,30,/api/v1/teams/30,Minnesota Wild,Ryan,Ryan Suter,"6' 2""",8470600,Suter,/api/v1/people/8470600,USA,20,D,D,Defenseman,Defenseman,False,Y,L,[{'type': {'displayName': 'statsSingleSeason'}...,208


Everything was successfully flattened into a single row except for the 'stats' column, which is nestled in another dictionary within a list. This column needs to be flattened again with json_normalize then joined with our existing columns.

In [9]:
stats = json_normalize(example_row['stats'][0][0]['splits'])
example_row = example_row.join(stats)
example_row

,active,alternateCaptain,birthCity,birthCountry,birthDate,birthStateProvince,captain,currentAge,currentTeam.id,currentTeam.link,currentTeam.name,firstName,fullName,height,id,lastName,link,nationality,primaryNumber,primaryPosition.abbreviation,primaryPosition.code,primaryPosition.name,primaryPosition.type,rookie,rosterStatus,shootsCatches,stats,weight,season,stat.assists,stat.blocked,stat.evenTimeOnIce,stat.evenTimeOnIcePerGame,stat.faceOffPct,stat.gameWinningGoals,stat.games,stat.goals,stat.hits,stat.overTimeGoals,stat.penaltyMinutes,stat.pim,stat.plusMinus,stat.points,stat.powerPlayGoals,stat.powerPlayPoints,stat.powerPlayTimeOnIce,stat.powerPlayTimeOnIcePerGame,stat.shifts,stat.shortHandedGoals,stat.shortHandedPoints,stat.shortHandedTimeOnIce,stat.shortHandedTimeOnIcePerGame,stat.shotPct,stat.shots,stat.timeOnIce,stat.timeOnIcePerGame
0,True,True,Madison,USA,1985-01-21,WI,False,33,30,/api/v1/teams/30,Minnesota Wild,Ryan,Ryan Suter,"6' 2""",8470600,Suter,/api/v1/people/8470600,USA,20,D,D,Defenseman,Defenseman,False,Y,L,[{'type': {'displayName': 'statsSingleSeason'}...,208,20182019,5,18,165:51,20:43,0.0,1,8,2,4,0,6,6,2,7,0,1,16:46,02:05,218,1,1,23:24,02:55,15.4,13,206:01,25:45


The final output above is all the possible data we can get from the API link for each player. We can do some clean-up by using the 'drop' function on any columns that we do not need (NOTE: Dropped columns are arbitrary, you can drop whichever ones you want).

In [10]:
example_row.drop(['currentTeam.link', 'link', 'stats', 'season'], axis=1, inplace=True)
example_row

,active,alternateCaptain,birthCity,birthCountry,birthDate,birthStateProvince,captain,currentAge,currentTeam.id,currentTeam.name,firstName,fullName,height,id,lastName,nationality,primaryNumber,primaryPosition.abbreviation,primaryPosition.code,primaryPosition.name,primaryPosition.type,rookie,rosterStatus,shootsCatches,weight,stat.assists,stat.blocked,stat.evenTimeOnIce,stat.evenTimeOnIcePerGame,stat.faceOffPct,stat.gameWinningGoals,stat.games,stat.goals,stat.hits,stat.overTimeGoals,stat.penaltyMinutes,stat.pim,stat.plusMinus,stat.points,stat.powerPlayGoals,stat.powerPlayPoints,stat.powerPlayTimeOnIce,stat.powerPlayTimeOnIcePerGame,stat.shifts,stat.shortHandedGoals,stat.shortHandedPoints,stat.shortHandedTimeOnIce,stat.shortHandedTimeOnIcePerGame,stat.shotPct,stat.shots,stat.timeOnIce,stat.timeOnIcePerGame
0,True,True,Madison,USA,1985-01-21,WI,False,33,30,Minnesota Wild,Ryan,Ryan Suter,"6' 2""",8470600,Suter,USA,20,D,D,Defenseman,Defenseman,False,Y,L,208,5,18,165:51,20:43,0.0,1,8,2,4,0,6,6,2,7,0,1,16:46,02:05,218,1,1,23:24,02:55,15.4,13,206:01,25:45


You can further pretty the data by re-ordering the columns. (NOTE: Re-organized columns are also arbitray, feel free to re-order the columns any way you want as well).

In [12]:
example_row = example_row[["id", "firstName", "lastName", "currentAge", "active", "alternateCaptain", "birthCity",
                "birthCountry", "birthDate", "birthStateProvince",
                "captain", "currentTeam.id", "currentTeam.name", "fullName", "height",
                "nationality", "primaryNumber", "primaryPosition.abbreviation", 
                "primaryPosition.code", "primaryPosition.name", "primaryPosition.type", "rookie", "rosterStatus",
                "shootsCatches", "weight", "stat.assists", "stat.blocked", "stat.evenTimeOnIce", 
                "stat.evenTimeOnIcePerGame", "stat.faceOffPct", "stat.gameWinningGoals", "stat.games", 
                "stat.goals", "stat.hits", "stat.overTimeGoals", "stat.penaltyMinutes", "stat.pim", 
                "stat.plusMinus", "stat.points", "stat.powerPlayGoals", "stat.powerPlayPoints", 
                "stat.powerPlayTimeOnIce", "stat.powerPlayTimeOnIcePerGame", "stat.shifts", 
                "stat.shortHandedGoals", "stat.shortHandedPoints", "stat.shortHandedTimeOnIce",
                "stat.shortHandedTimeOnIcePerGame", "stat.shotPct", "stat.shots", "stat.timeOnIce", 
                "stat.timeOnIcePerGame"]]

example_row.head()

,id,firstName,lastName,currentAge,active,alternateCaptain,birthCity,birthCountry,birthDate,birthStateProvince,captain,currentTeam.id,currentTeam.name,fullName,height,nationality,primaryNumber,primaryPosition.abbreviation,primaryPosition.code,primaryPosition.name,primaryPosition.type,rookie,rosterStatus,shootsCatches,weight,stat.assists,stat.blocked,stat.evenTimeOnIce,stat.evenTimeOnIcePerGame,stat.faceOffPct,stat.gameWinningGoals,stat.games,stat.goals,stat.hits,stat.overTimeGoals,stat.penaltyMinutes,stat.pim,stat.plusMinus,stat.points,stat.powerPlayGoals,stat.powerPlayPoints,stat.powerPlayTimeOnIce,stat.powerPlayTimeOnIcePerGame,stat.shifts,stat.shortHandedGoals,stat.shortHandedPoints,stat.shortHandedTimeOnIce,stat.shortHandedTimeOnIcePerGame,stat.shotPct,stat.shots,stat.timeOnIce,stat.timeOnIcePerGame
0,8470600,Ryan,Suter,33,True,True,Madison,USA,1985-01-21,WI,False,30,Minnesota Wild,Ryan Suter,"6' 2""",USA,20,D,D,Defenseman,Defenseman,False,Y,L,208,5,18,165:51,20:43,0.0,1,8,2,4,0,6,6,2,7,0,1,16:46,02:05,218,1,1,23:24,02:55,15.4,13,206:01,25:45


Our json-to-dataframe process is now done. Now this just needs to be combined into a single function that can be applied to every row.

In [13]:
def make_row(url):
    response = requests.get(url)
    row = json_normalize(response.json()['people'])
    stats = json_normalize(row['stats'][0][0]['splits'])
    row = row.join(stats)
    return row

# quickly just going to test the function on a random URL
make_row("http://statsapi.web.nhl.com/api/v1/people/8470600?hydrate=stats(splits=statsSingleSeason)")

,active,alternateCaptain,birthCity,birthCountry,birthDate,birthStateProvince,captain,currentAge,currentTeam.id,currentTeam.link,currentTeam.name,firstName,fullName,height,id,lastName,link,nationality,primaryNumber,primaryPosition.abbreviation,primaryPosition.code,primaryPosition.name,primaryPosition.type,rookie,rosterStatus,shootsCatches,stats,weight,season,stat.assists,stat.blocked,stat.evenTimeOnIce,stat.evenTimeOnIcePerGame,stat.faceOffPct,stat.gameWinningGoals,stat.games,stat.goals,stat.hits,stat.overTimeGoals,stat.penaltyMinutes,stat.pim,stat.plusMinus,stat.points,stat.powerPlayGoals,stat.powerPlayPoints,stat.powerPlayTimeOnIce,stat.powerPlayTimeOnIcePerGame,stat.shifts,stat.shortHandedGoals,stat.shortHandedPoints,stat.shortHandedTimeOnIce,stat.shortHandedTimeOnIcePerGame,stat.shotPct,stat.shots,stat.timeOnIce,stat.timeOnIcePerGame
0,True,True,Madison,USA,1985-01-21,WI,False,33,30,/api/v1/teams/30,Minnesota Wild,Ryan,Ryan Suter,"6' 2""",8470600,Suter,/api/v1/people/8470600,USA,20,D,D,Defenseman,Defenseman,False,Y,L,[{'type': {'displayName': 'statsSingleSeason'}...,208,20182019,5,18,165:51,20:43,0.0,1,8,2,4,0,6,6,2,7,0,1,16:46,02:05,218,1,1,23:24,02:55,15.4,13,206:01,25:45


Seems to look alright so now we can just apply our function to every row in our spreadsheet. We'll iterate through each row and generate our data using the URL provided (code below may take a few minutes).

In [16]:
final = make_row(nhl['url'][0])
for index, row in nhl[1:].iterrows():
    final = final.append(make_row(row["url"]), sort=False)

In [17]:
final.head()

,active,alternateCaptain,birthCity,birthCountry,birthDate,birthStateProvince,captain,currentAge,currentTeam.id,currentTeam.link,currentTeam.name,firstName,fullName,height,id,lastName,link,nationality,primaryNumber,primaryPosition.abbreviation,primaryPosition.code,primaryPosition.name,primaryPosition.type,rookie,rosterStatus,shootsCatches,stats,weight,season,stat.assists,stat.blocked,stat.evenTimeOnIce,stat.evenTimeOnIcePerGame,stat.faceOffPct,stat.gameWinningGoals,stat.games,stat.goals,stat.hits,stat.overTimeGoals,stat.penaltyMinutes,stat.pim,stat.plusMinus,stat.points,stat.powerPlayGoals,stat.powerPlayPoints,stat.powerPlayTimeOnIce,stat.powerPlayTimeOnIcePerGame,stat.shifts,stat.shortHandedGoals,stat.shortHandedPoints,stat.shortHandedTimeOnIce,stat.shortHandedTimeOnIcePerGame,stat.shotPct,stat.shots,stat.timeOnIce,stat.timeOnIcePerGame,stat.evenSaves,stat.evenShots,stat.evenStrengthSavePercentage,stat.gamesStarted,stat.goalAgainstAverage,stat.goalsAgainst,stat.losses,stat.ot,stat.powerPlaySavePercentage,stat.powerPlaySaves,stat.powerPlayShots,stat.savePercentage,stat.saves,stat.shortHandedSavePercentage,stat.shortHandedSaves,stat.shortHandedShots,stat.shotsAgainst,stat.shutouts,stat.ties,stat.wins
0,True,True,Madison,USA,1985-01-21,WI,False,33.0,30.0,/api/v1/teams/30,Minnesota Wild,Ryan,Ryan Suter,"6' 2""",8470600,Suter,/api/v1/people/8470600,USA,20,D,D,Defenseman,Defenseman,False,Y,L,[{'type': {'displayName': 'statsSingleSeason'}...,208,20182019,5.0,18.0,165:51,20:43,0.0,1.0,8.0,2.0,4.0,0.0,6,6.0,2.0,7.0,0.0,1.0,16:46,02:05,218.0,1.0,1.0,23:24,02:55,15.4,13.0,206:01,25:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,True,True,Borlange,SWE,1990-05-24,NaN,False,28.0,18.0,/api/v1/teams/18,Nashville Predators,Mattias,Mattias Ekholm,"6' 4""",8475218,Ekholm,/api/v1/people/8475218,SWE,14,D,D,Defenseman,Defenseman,False,Y,L,[{'type': {'displayName': 'statsSingleSeason'}...,215,20182019,5.0,13.0,170:19,18:55,0.0,0.0,9.0,2.0,8.0,0.0,7,7.0,8.0,7.0,0.0,0.0,03:52,00:25,255.0,0.0,1.0,22:27,02:29,12.5,16.0,196:38,21:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,True,False,Montreal,CAN,1987-03-30,QC,False,31.0,28.0,/api/v1/teams/28,San Jose Sharks,Marc-Edouard,Marc-Edouard Vlasic,"6' 1""",8471709,Vlasic,/api/v1/people/8471709,CAN,44,D,D,Defenseman,Defenseman,False,Y,L,[{'type': {'displayName': 'statsSingleSeason'}...,205,20182019,3.0,14.0,162:24,18:02,0.0,0.0,9.0,0.0,3.0,0.0,0,0.0,1.0,3.0,0.0,0.0,12:58,01:26,253.0,0.0,0.0,36:18,04:02,0.0,22.0,211:40,23:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,True,False,Denver,USA,1994-05-01,CO,False,24.0,12.0,/api/v1/teams/12,Carolina Hurricanes,Jaccob,Jaccob Slavin,"6' 3""",8476958,Slavin,/api/v1/people/8476958,USA,74,D,D,Defenseman,Defenseman,False,Y,L,[{'type': {'displayName': 'statsSingleSeason'}...,205,20182019,4.0,14.0,160:05,17:47,0.0,0.0,9.0,0.0,4.0,0.0,0,0.0,-1.0,4.0,0.0,0.0,21:51,02:25,242.0,0.0,1.0,24:03,02:40,0.0,23.0,205:59,22:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,True,False,New Rochelle,USA,1989-01-29,NY,False,29.0,3.0,/api/v1/teams/3,New York Rangers,Kevin,Kevin Shattenkirk,"6' 0""",8474031,Shattenkirk,/api/v1/people/8474031,USA,22,D,D,Defenseman,Defenseman,False,Y,R,[{'type': {'displayName': 'statsSingleSeason'}...,206,20182019,2.0,18.0,117:38,14:42,0.0,0.0,8.0,0.0,4.0,0.0,0,0.0,-4.0,2.0,0.0,1.0,24:10,03:01,179.0,0.0,0.0,00:02,00:00,0.0,18.0,141:50,17:43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can organize our code as we see fit (dropping uncessary columns, re-organizing columns into desirable order, etc)

In [18]:
final.drop(['currentTeam.link', 'link', 'stats', 'season'], axis=1, inplace=False)
    
final = final[["id", "firstName", "lastName", "currentAge", "active", "alternateCaptain", "birthCity",
                "birthCountry", "birthDate", "birthStateProvince",
                "captain", "currentTeam.id", "currentTeam.name", "fullName", "height",
                "nationality", "primaryNumber", "primaryPosition.abbreviation", 
                "primaryPosition.code", "primaryPosition.name", "primaryPosition.type", "rookie", "rosterStatus",
                "shootsCatches", "weight", "stat.assists", "stat.blocked", "stat.evenTimeOnIce", 
                "stat.evenTimeOnIcePerGame", "stat.faceOffPct", "stat.gameWinningGoals", "stat.games", 
                "stat.goals", "stat.hits", "stat.overTimeGoals", "stat.penaltyMinutes", "stat.pim", 
                "stat.plusMinus", "stat.points", "stat.powerPlayGoals", "stat.powerPlayPoints", 
                "stat.powerPlayTimeOnIce", "stat.powerPlayTimeOnIcePerGame", "stat.shifts", 
                "stat.shortHandedGoals", "stat.shortHandedPoints", "stat.shortHandedTimeOnIce",
                "stat.shortHandedTimeOnIcePerGame", "stat.shotPct", "stat.shots", "stat.timeOnIce", 
                "stat.timeOnIcePerGame"]]

In [20]:
final.head()

,id,firstName,lastName,currentAge,active,alternateCaptain,birthCity,birthCountry,birthDate,birthStateProvince,captain,currentTeam.id,currentTeam.name,fullName,height,nationality,primaryNumber,primaryPosition.abbreviation,primaryPosition.code,primaryPosition.name,primaryPosition.type,rookie,rosterStatus,shootsCatches,weight,stat.assists,stat.blocked,stat.evenTimeOnIce,stat.evenTimeOnIcePerGame,stat.faceOffPct,stat.gameWinningGoals,stat.games,stat.goals,stat.hits,stat.overTimeGoals,stat.penaltyMinutes,stat.pim,stat.plusMinus,stat.points,stat.powerPlayGoals,stat.powerPlayPoints,stat.powerPlayTimeOnIce,stat.powerPlayTimeOnIcePerGame,stat.shifts,stat.shortHandedGoals,stat.shortHandedPoints,stat.shortHandedTimeOnIce,stat.shortHandedTimeOnIcePerGame,stat.shotPct,stat.shots,stat.timeOnIce,stat.timeOnIcePerGame
0,8470600,Ryan,Suter,33.0,True,True,Madison,USA,1985-01-21,WI,False,30.0,Minnesota Wild,Ryan Suter,"6' 2""",USA,20,D,D,Defenseman,Defenseman,False,Y,L,208,5.0,18.0,165:51,20:43,0.0,1.0,8.0,2.0,4.0,0.0,6,6.0,2.0,7.0,0.0,1.0,16:46,02:05,218.0,1.0,1.0,23:24,02:55,15.4,13.0,206:01,25:45
0,8475218,Mattias,Ekholm,28.0,True,True,Borlange,SWE,1990-05-24,NaN,False,18.0,Nashville Predators,Mattias Ekholm,"6' 4""",SWE,14,D,D,Defenseman,Defenseman,False,Y,L,215,5.0,13.0,170:19,18:55,0.0,0.0,9.0,2.0,8.0,0.0,7,7.0,8.0,7.0,0.0,0.0,03:52,00:25,255.0,0.0,1.0,22:27,02:29,12.5,16.0,196:38,21:50
0,8471709,Marc-Edouard,Vlasic,31.0,True,False,Montreal,CAN,1987-03-30,QC,False,28.0,San Jose Sharks,Marc-Edouard Vlasic,"6' 1""",CAN,44,D,D,Defenseman,Defenseman,False,Y,L,205,3.0,14.0,162:24,18:02,0.0,0.0,9.0,0.0,3.0,0.0,0,0.0,1.0,3.0,0.0,0.0,12:58,01:26,253.0,0.0,0.0,36:18,04:02,0.0,22.0,211:40,23:31
0,8476958,Jaccob,Slavin,24.0,True,False,Denver,USA,1994-05-01,CO,False,12.0,Carolina Hurricanes,Jaccob Slavin,"6' 3""",USA,74,D,D,Defenseman,Defenseman,False,Y,L,205,4.0,14.0,160:05,17:47,0.0,0.0,9.0,0.0,4.0,0.0,0,0.0,-1.0,4.0,0.0,0.0,21:51,02:25,242.0,0.0,1.0,24:03,02:40,0.0,23.0,205:59,22:53
0,8474031,Kevin,Shattenkirk,29.0,True,False,New Rochelle,USA,1989-01-29,NY,False,3.0,New York Rangers,Kevin Shattenkirk,"6' 0""",USA,22,D,D,Defenseman,Defenseman,False,Y,R,206,2.0,18.0,117:38,14:42,0.0,0.0,8.0,0.0,4.0,0.0,0,0.0,-4.0,2.0,0.0,1.0,24:10,03:01,179.0,0.0,0.0,00:02,00:00,0.0,18.0,141:50,17:43


And that's it. Export to a .csv or Excel file as you see fit.

In [21]:
final.to_csv("nhl_stats.csv", index=False)